In [1]:
import json

from rightmove_webscraper import RightmoveData

In [104]:
url = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E1271&sortType=6&propertyTypes=detached%2Csemi-detached%2Cterraced&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="
rm = RightmoveData(url)

In [105]:
rm.get_results.head()

,price,type,address,url,agent_url,postcode,full_postcode,number_bedrooms,search_date
0,85000.0,2 bedroom terraced house for sale,"Coronation Road, Hartshill",http://www.rightmove.co.uk/properties/12092584...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,2.0,2022-08-15 17:29:03.752358
1,1275000.0,4 bedroom detached house for sale,"Park Drive, Stoke-on-Trent, ST4",http://www.rightmove.co.uk/properties/12608611...,http://www.rightmove.co.uk/estate-agents/agent...,ST4,NaN,4.0,2022-08-15 17:29:03.752358
2,795000.0,5 bedroom detached house for sale,"Stanley Road, Stanley Moor, ST9",http://www.rightmove.co.uk/properties/81589452...,http://www.rightmove.co.uk/estate-agents/agent...,ST9,NaN,5.0,2022-08-15 17:29:03.752358
3,795000.0,5 bedroom detached house for sale,"Edgefields Lane, Stockton Brook, ST9",http://www.rightmove.co.uk/properties/12592475...,http://www.rightmove.co.uk/estate-agents/agent...,ST9,NaN,5.0,2022-08-15 17:29:03.752358
4,795000.0,6 bedroom detached house for sale,"Ball Lane, Brown Edge, Staffordshire, ST6",http://www.rightmove.co.uk/properties/11429797...,http://www.rightmove.co.uk/estate-agents/agent...,ST6,NaN,6.0,2022-08-15 17:29:03.752358


In [106]:
rm.get_results

,price,type,address,url,agent_url,postcode,full_postcode,number_bedrooms,search_date
0,85000.0,2 bedroom terraced house for sale,"Coronation Road, Hartshill",http://www.rightmove.co.uk/properties/12092584...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,2.0,2022-08-15 17:29:03.752358
1,1275000.0,4 bedroom detached house for sale,"Park Drive, Stoke-on-Trent, ST4",http://www.rightmove.co.uk/properties/12608611...,http://www.rightmove.co.uk/estate-agents/agent...,ST4,NaN,4.0,2022-08-15 17:29:03.752358
2,795000.0,5 bedroom detached house for sale,"Stanley Road, Stanley Moor, ST9",http://www.rightmove.co.uk/properties/81589452...,http://www.rightmove.co.uk/estate-agents/agent...,ST9,NaN,5.0,2022-08-15 17:29:03.752358
3,795000.0,5 bedroom detached house for sale,"Edgefields Lane, Stockton Brook, ST9",http://www.rightmove.co.uk/properties/12592475...,http://www.rightmove.co.uk/estate-agents/agent...,ST9,NaN,5.0,2022-08-15 17:29:03.752358
4,795000.0,6 bedroom detached house for sale,"Ball Lane, Brown Edge, Staffordshire, ST6",http://www.rightmove.co.uk/properties/11429797...,http://www.rightmove.co.uk/estate-agents/agent...,ST6,NaN,6.0,2022-08-15 17:29:03.752358
...,...,...,...,...,...,...,...,...,...
605,15000.0,Land for sale,"Grindley Lane, Blythe Bridge",http://www.rightmove.co.uk/properties/12089837...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,NaN,2022-08-15 17:29:03.752358
606,15000.0,2 bedroom terraced house for sale,"Patterdale Street, Stoke-on-Trent, Staffordshi...",http://www.rightmove.co.uk/properties/12586683...,http://www.rightmove.co.uk/estate-agents/agent...,ST6,NaN,2.0,2022-08-15 17:29:03.752358
607,10000.0,2 bedroom terraced house for sale,"432A King Street, Stoke-on-Trent, ST4 3DB",http://www.rightmove.co.uk/properties/12599881...,http://www.rightmove.co.uk/estate-agents/agent...,ST4,ST4 3DB,2.0,2022-08-15 17:29:03.752358
608,10000.0,Land for sale,"40 Market Street, Longton, Stoke-On-Trent ST3 1BS",http://www.rightmove.co.uk/properties/12550649...,None,ST3,ST3 1BS,NaN,2022-08-15 17:29:03.752358


In [81]:
path = 'city_names.txt'
cities = []

In [82]:
with open(path) as file:
    for line in file:
        cities.append(line.replace("\n", ""))

In [83]:
cities

['Bath',
 'Birmingham',
 'Bradford',
 'Brighton and Hove',
 'Bristol',
 'Cambridge',
 'Canterbury',
 'Carlisle',
 'Chelmsford',
 'Chester',
 'Chichester',
 'Colchester',
 'Coventry',
 'Derby',
 'Doncaster',
 'Douglas',
 'Durham',
 'Ely',
 'Exeter',
 'Gloucester',
 'Hereford',
 'Kingston upon Hull',
 'Lancaster',
 'Leeds',
 'Leicester',
 'Lichfield',
 'Lincoln',
 'Liverpool',
 'London',
 'Manchester',
 'Milton Keynes',
 'Newcastle upon Tyne',
 'Norwich',
 'Nottingham',
 'Oxford',
 'Peterborough',
 'Plymouth',
 'Portsmouth',
 'Preston',
 'Ripon',
 'Salford',
 'Salisbury',
 'Sheffield',
 'Southampton',
 'Southend-On-Sea',
 'St Albans',
 'Stoke-on-Trent',
 'Sunderland',
 'Truro',
 'Wakefield',
 'Wells',
 'Westminster',
 'Winchester',
 'Wolverhampton',
 'Worcester',
 'York']

In [100]:
city_encodings = {}

In [101]:
import requests, json
import re
path = 'https://www.rightmove.co.uk/typeAhead/uknostreet/'

In [102]:
for city in cities:
    count = 0
    for char in city:
        if count == 2:
            path += '/'
            count = 0
        if char == "-" or char == " ":
            path += '%20'
        else:
            path += char.upper()
        count += 1

    print(path)
    res = requests.get(path).json()
    out = json.dumps(res, indent=2)
    p = re.search('REGION\^(.+?),', out)
    if p:
        found = p.group(1)
    city_encodings[city] = found.rstrip('"')
    path = 'https://www.rightmove.co.uk/typeAhead/uknostreet/'



https://www.rightmove.co.uk/typeAhead/uknostreet/BA/TH
https://www.rightmove.co.uk/typeAhead/uknostreet/BI/RM/IN/GH/AM
https://www.rightmove.co.uk/typeAhead/uknostreet/BR/AD/FO/RD
https://www.rightmove.co.uk/typeAhead/uknostreet/BR/IG/HT/ON/%20A/ND/%20H/OV/E
https://www.rightmove.co.uk/typeAhead/uknostreet/BR/IS/TO/L
https://www.rightmove.co.uk/typeAhead/uknostreet/CA/MB/RI/DG/E
https://www.rightmove.co.uk/typeAhead/uknostreet/CA/NT/ER/BU/RY
https://www.rightmove.co.uk/typeAhead/uknostreet/CA/RL/IS/LE
https://www.rightmove.co.uk/typeAhead/uknostreet/CH/EL/MS/FO/RD
https://www.rightmove.co.uk/typeAhead/uknostreet/CH/ES/TE/R
https://www.rightmove.co.uk/typeAhead/uknostreet/CH/IC/HE/ST/ER
https://www.rightmove.co.uk/typeAhead/uknostreet/CO/LC/HE/ST/ER
https://www.rightmove.co.uk/typeAhead/uknostreet/CO/VE/NT/RY
https://www.rightmove.co.uk/typeAhead/uknostreet/DE/RB/Y
https://www.rightmove.co.uk/typeAhead/uknostreet/DO/NC/AS/TE/R
https://www.rightmove.co.uk/typeAhead/uknostreet/DO/UG/LA/S


In [103]:
city_encodings

{'Bath': '116',
 'Birmingham': '162',
 'Bradford': '198',
 'Brighton and Hove': '93554',
 'Bristol': '219',
 'Cambridge': '274',
 'Canterbury': '279',
 'Carlisle': '283',
 'Chelmsford': '307',
 'Chester': '313',
 'Chichester': '316',
 'Colchester': '347',
 'Coventry': '368',
 'Derby': '418',
 'Doncaster': '430',
 'Douglas': '8346',
 'Durham': '460',
 'Ely': '483',
 'Exeter': '494',
 'Gloucester': '556',
 'Hereford': '61304',
 'Kingston upon Hull': '746',
 'Lancaster': '769',
 'Leeds': '787',
 'Leicester': '789',
 'Lichfield': '800',
 'Lincoln': '804',
 'Liverpool': '813',
 'London': '87490',
 'Manchester': '904',
 'Milton Keynes': '940',
 'Newcastle upon Tyne': '984',
 'Norwich': '1018',
 'Nottingham': '1019',
 'Oxford': '1036',
 'Peterborough': '1061',
 'Plymouth': '1073',
 'Portsmouth': '1089',
 'Preston': '1097',
 'Ripon': '1132',
 'Salford': '1164',
 'Salisbury': '1165',
 'Sheffield': '1195',
 'Southampton': '1231',
 'Southend-On-Sea': '1232',
 'St Albans': '94262',
 'Stoke-on-Tren

In [115]:
path = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E1271&sortType=6&propertyTypes=detached%2Csemi-detached%2Cterraced&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="

In [144]:
path.find("REGION%")
encoding = city_encodings["York"]
old = path[path.index("REGION%"): path.index("&")]
new = path.replace(old, "REGION%" + encoding)
new

'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%1498&sortType=6&propertyTypes=detached%2Csemi-detached%2Cterraced&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords='

In [145]:
rm = RightmoveData(new)
rm.get_results.head()

IndexError: list index out of range